In [6]:
!pip install datasets
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install tensorflow

In [9]:
import os
from datasets import load_dataset
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image